In [ ]:
from database import get_dataframe_from_sql_query
from datetime import datetime as dt, timedelta as td
from constant import BEEXP_SCRIPT, ORDER_ITEM_DATE_COLS
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update
def get_beexp_since(ordered_date):
    oders_df = get_dataframe_from_sql_query(
    sql_script=BEEXP_SCRIPT,
    date_cols=ORDER_ITEM_DATE_COLS
    )
    mask = (oders_df['dt_implant'] >= ordered_date)
    return oders_df.loc[mask]

beexp_df = get_beexp_since(current_update)
products = [f"Quantidade de Ingressos: [dado] Nome do Vendedor: [dado] Tipo de Ingresso: [dado] Valor faturado: [dado] Forma de pagamento: [dado] Código do cliente: [dado] Data da venda: [dado] E-mail do representante: [dado] O respectivo vendedor já recebeu o link de cadastro do participante." for cod in ]
send_messages_by_group(
        gdrive_folder_id=products, group='comercial'
)

In [ ]:
script = 'data/in/beexp_orders.sql'
from database import get_dataframe_from_sql
beexp_df = get_dataframe_from_sql(script)     
beexp_filtered_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]

In [ ]:
from mkt_automation import get_beexp_since, get_beexp_manager_message
from datetime import datetime as dt, timedelta as td
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update

beexp_df = get_beexp_since(current_update)
beexp_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]
for row in beexp_df.itertuples():
  beexp_message = get_beexp_manager_message(row)
  send_messages_by_group(gdrive_folder_id=beexp_message, group='beexp_manager')